# Data analyses Crane datasets

## Contents notebook
- Import required modules
- Define generic functions
- Per Country:
  - Import sets
  - Basic analysis of the first Crane dataset
  - Basic analysis of remaining Crane datasets
  - Visualize Crane datasets
  - Filter Crane datasets
  - Export Crane datasets to JSON
  - Export Crane datasets to GeoJSON
- Compare German and Swedish crane Datasets
  - Compare  sets

---
## Note 
- The data analyses of the first dataset(per country) will be done manually. The rest of the datasets(per country), will be analysed using the generic function:"basic_analyses()"
- The GeoJSON data will not be used during the creation of the application. The reason it's describe how this is done, is because it can be usefull in other cases. Converting fileformats to GeoJSON can also be done using geopandas(See Cookbook: ETL-Process with datasets)

## Datasets used in notebook. ("../Data/Crane_CSV/")
- Swedish Crane datasets
    - 20181003_Dataset_SV_TrackerID_9381_ColorCode_RRW-BuGBk_Crane_Frida.csv
    - 20181003_Dataset_SV_TrackerID_9407_ColorCode_RRW-BuGBY_Crane_Agnetha.csv
    - 20181003_Dataset_SV_TrackerID_9472_ColorCode_RRW-BuGR_Crane_Cajsa.csv

- German Crane datasets
    - 20180928_Dataset_DE_GPS_Crane_181527_iCora_Crane_13_BuBuBr-YBuBk.csv
    - 20191103_Dataset_DE_GPS_Crane_181528_iCora_Crane_15_BuBuBr-WYW_Lotta.csv
 
- Lithuanian Crane datasets
    - 20200103_Dataset_LT_TrackerID_16121_Crane_Lita.csv
  
    
    

---
## Import required modules
---

In [ ]:
import pandas as pd
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature 
import matplotlib.pyplot as plt

---
## Define generic functions

---
### Function: create_plot()
This function creates a plot from the inserted data on the map.

- parameter 1: The dataframe you want to plot
- parameter 2: The longitude column in the dataframe
- parameter 3: The latitude column in the dataframe

In [ ]:
def create_plot(df,lonColumn,latColumn):
    return plt.plot(df[lonColumn], df[latColumn])

---
### Function: init_cartopymap():
This function creates a new cartopyMap and then returns the map instance

In [ ]:
def init_cartopymap():
    
    plt.figure(figsize = (20, 12))
    
    m = plt.axes(projection=ccrs.PlateCarree())

    m.coastlines(resolution='10m')
    
    m.add_feature(cartopy.feature.LAND.with_scale('10m'), edgecolor='black', facecolor = "white")
    
    m.add_feature(cfeature.OCEAN)
    
    m.add_feature(cfeature.LAKES.with_scale('10m'), edgecolor = 'black')
    
    m.add_feature(cfeature.RIVERS.with_scale('10m')) 
    
    m.add_feature(cfeature.BORDERS.with_scale('10m'))

    return m 

---
### Function: dfs_on_cartopymap():
This function creates a map and visualizes the inserted dataframes on a CartopyMap

- parameter 1: A list of dataframes, this can also be a list of just one dataframe
- parameter 2: The longitude column in the dataframe
- parameter 3: The latitude column in the dataframe
- parameter 4: The color of the datapoints
- parameter 5: The size of the datapoints

In [ ]:
def dfs_on_cartopymap(dfList,lonColumn,latColumn,color,size):

    m = init_cartopymap()
    
    for df in dfList: 
        
        m.scatter(df[lonColumn], 
                  df[latColumn],
                  color=color, 
                  s = size)    
    return m

---
### Function: compare_sets()
This function compares the columns of 2 datasets and returns "are equal" if the columns of both datasets are the same. <br> The function returns the differences if the columns of both sets are not equal.

- Parameter 1 = the first set you want to compare<br>
- Parameter 2 = the second set you want to compare

In [ ]:
def compare_sets(set1,set2):
    
    # Create an empty list to append differences to
    diffs_set2=["IN SWEDISH BUT NOT IN GERMAN:",'']
    diffs_set1 =["IN GERMAN BUT NOT IN SWEDISH:",'']
    
    # Create an empty list to append equalities to
    eqs = ["IN BOTH SETS:",'']
    
    # Check is sets are the same
    # If True return "sets are equal"
    if set1.equals(set2):
        return "Sets are equal"
        
    # Compare sets and append diffs and eqs to corresponding lists
    else:
        for x in set1:
            if x in set2:
                eqs.append(x)
            if x not in set2:
                diffs_set1.append(x)
        for x in set2:
            if x not in set1:
                diffs_set2.append(x)
    return diffs_set1, diffs_set2,eqs

---
### Function: basic_analysis()
This function performs a basic data analyses on an CSV file. <br>
- Parameter 1 = The file location of the CSV file

In [ ]:
def basic_analysis(file_input):
    df = pd.read_csv(file_input)
    
    print('\n--------------------------------BASIC INFORMATION-------------------------------\n\n')
    
    print("Dataframe length: " + str(len(df))+'\n') 
    
    print("Dataframe columns:" + str(len(df.columns))+'\n')
    
    print("Start date of study :" + str(df[:1].timestamp)+'\n')
    
    print("Start position (latitude) :" + str(df[:1]['location-lat'])+'\n')
    
    print("Start position (longitude) :" + str(df[:1]['location-long'])+'\n')
    
    print("End date of study :"  + str(df[-1:].timestamp)+'\n')
    
    print("End position (latitude) :" + str(df[-1:]['location-lat'])+'\n')
    
    print("End position (longitude) :" + str(df[-1:]['location-long'])+'\n')
    
    print('\n------------------------------COLUMNS AND DATA TYPES----------------------------\n\n')
    
    print(df.dtypes)
    
    print('\n------------------------------FIRST ROW OF DATAFRAME----------------------------\n\n')
    
    return df

---
# Swedish Cranes

Attaching the transmitters was done in a place called Grimsås, Sweden. 

Some info about the data set:
- The tracker contains a GPS sensor and a few other environmental sensors, of which the sensor data every 15 minutes (4x / hour) if an SMS message is sent via a GSM channel sent. All that sensor data can be found in the logbook.
- The color ring code of this crane is RRW-BuGBk and can be found in the column Search Studies by clicking on the small black web link Implementations, along with its tracker ID and the ring number of the metal ring with which the bird is ringed.
- Details: the most famous location is North-East France.

This datasets are obtained from https://www.movebank.org/. 

---
## Agnetha
### A Swedish Crane

---

#### Read the CSV dataset using Pandas.

In [ ]:
Agnetha = pd.read_csv(
    '../../datasets/CSV/20181003_Dataset_SV_TrackerID_9407_ColorCode_RRW-BuGBY_Crane_Agnetha.csv')

#### Print the amount of datarows (Transmissions).

In [ ]:
len(Agnetha)

#### Print the amount of columns in the dataset.

In [ ]:
len(Agnetha.columns)

#### Print information related to the start position / date.

In [ ]:
print("Start date of study :" + str(Agnetha[:1].timestamp)+'\n')
print("Start position (latitude) :" + str(Agnetha[:1]['location-lat'])+'\n')
print("Start position (longitude) :" + str(Agnetha[:1]['location-long'])+'\n')

#### Print information related to the end position / date.

In [ ]:
print("End date of study :"  + str(Agnetha[-1:].timestamp)+'\n')
print("End position (latitude) :" + str(Agnetha[-1:]['location-lat'])+'\n')
print("End position (longitude) :" + str(Agnetha[-1:]['location-long'])+'\n')

#### Show first row of dataframe.

In [ ]:
Agnetha[:1]

#### Print columnames and their datatypes.

In [ ]:
Agnetha.dtypes

#### Create a table and put the values, obtained from the line above, in that table.

#### Columns with their data type (19): 

|Column|Type|Desc.|
|--|--|--|
|event-id | int| The ID of the transmission (Unique) |
|visible|bool | Is the tracker still visible or not? | 
|timestamp |datetime| The DateTime of transmission |
|location-long| float|The Longitude location |
|location-lat | float| The Latitude location |
|argos:altitude|float | Argos type sensor for height of the transmission |
|gps:fix-type | int| Type of GPS Tracker |
|gps:hdop| float| Horizontal dilution of precision|
|gps:vdop| float| Vertical dilution of precision |
|ground-speed|float | Speed of the bird while on the ground. |
|heading | float| Direction the bird is faceing (In degrees) |
|height-above-ellipsoid | float| Height of the bird HAE -> MSL |
|tag-tech-spec |float | |
|tag-voltage|float | Voltage level of the tracker. |
|sensor-type|string | Type of sensor used in tracker. |
|individual-taxon-canonical-name | string|Name of the bird, in latin. |
|tag-local-identifier |int |ID of the tracker.  |
|individual-local-identifier |int |ID of the bird |
|study-name | string|Name of the study |


#### Create a plot using MatplotLib.

In [ ]:
create_plot(Agnetha,'location-long','location-lat') 

#### Create a plot MatplotLib on CartopyMap.

In [ ]:
dfs_on_cartopymap([Agnetha],'location-long','location-lat','red',1)

---
## Frida
### A Swedish Crane 

---

#### Perform basic analyses of Crane dataset and assign the returned dataframe to a variable.

In [ ]:
Frida = basic_analysis(
    '../../datasets/CSV/20181003_Dataset_SV_TrackerID_9381_ColorCode_RRW-BuGBk_Crane_Frida.csv')
Frida

#### Create a plot using MatplotLib.

In [ ]:
create_plot(Frida,'location-long','location-lat') 

#### Create a plot MatplotLib on CartopyMap.

In [ ]:
dfs_on_cartopymap([Frida],'location-long','location-lat','green',1)

---
## Cajsa
### A Swedish Crane 

---

#### Perform basic analyses of Crane dataset and assign the returned dataframe to a variable.

In [ ]:
Cajsa = basic_analysis(
    '../../datasets/CSV/20181003_Dataset_SV_TrackerID_9472_ColorCode_RRW-BuGR_Crane_Cajsa.csv')
Cajsa

#### Create a plot using MatplotLib.

In [ ]:
create_plot(Cajsa,'location-long','location-lat') 

#### Create a plot MatplotLib on CartopyMap.

In [ ]:
dfs_on_cartopymap([Cajsa],'location-long','location-lat','blue',1)

---
### Compare the datasets to check if all the columns are the same.
---

In [ ]:
print(compare_sets(Cajsa.columns,Frida.columns))
print(compare_sets(Cajsa.columns,Agnetha.columns))
print(compare_sets(Frida.columns,Agnetha.columns))

---
### Visualize all Swedish Crane datasets on a CartopyMap
---

In [ ]:
#Create the plot and assign size of the figure
plt.figure(figsize = (20, 12))

# Create the axes of the plot and assign the projection
m = plt.axes(projection=ccrs.PlateCarree())

# Create high resolution for coastal line
m.coastlines(resolution='10m')

# Add the land with a black edge color and white fill
m.add_feature(cartopy.feature.LAND.with_scale('10m'), edgecolor='black', facecolor = "white")

# Add the sea 
m.add_feature(cfeature.OCEAN) 

# Add lakes with a black edge color
m.add_feature(cfeature.LAKES.with_scale('10m'), edgecolor = 'black')

# Add rivers
m.add_feature(cfeature.RIVERS.with_scale('10m')) 

# Add borders
m.add_feature(cfeature.BORDERS.with_scale('10m'))

# Add the first Swedish Crane set
m.scatter(Agnetha['location-long'], Agnetha['location-lat'],color="red", s = 1)

# Add the second Swedish Crane set
m.scatter(Frida['location-long'], Frida['location-lat'],color="blue", s = 1)

# Add the Third Swedish Crane set
m.scatter(Cajsa['location-long'], Cajsa['location-lat'],color="green", s = 1)

---
### Filter Swedish crane dataset

In [ ]:
columns_to_filter = ['event-id', 'study-name',
                     'timestamp','visible',
                     'ground-speed','heading',
                     'location-long','location-lat',
                     'height-above-ellipsoid',
                     'individual-taxon-canonical-name',
                     'sensor-type','tag-voltage',                                          
                     'individual-local-identifier']

Filtered_Frida = Frida[columns_to_filter]

Filtered_Cajsa = Cajsa[columns_to_filter]

Filtered_Agnetha = Agnetha[columns_to_filter]

---
### Export Swedish Crane datasets to JSON
Create a json file en writing the contents of the dataframe using to_json() function.

- Parameter 1: filename of file that will be created
- Parameter 2: Orientation of the data. <br>

orientation "records" makes list like [{column -> value},{column -> value},....]

---

In [ ]:
Filtered_Frida.to_json(
    '../../datasets/JSON/Crane_JSON/Frida-SW.json',
    orient = 'records')

Filtered_Cajsa.to_json(
    '../../datasets/JSON/Crane_JSON/Cajsa-SW.json',
    orient = 'records')

Filtered_Agnetha.to_json(
    '../../datasets/JSON/Crane_JSON/Agnetha-SW.json',
    orient = 'records')

In [ ]:
print(compare_sets(Cajsa.columns,Frida.columns))
print(compare_sets(Cajsa.columns,Agnetha.columns))
print(compare_sets(Frida.columns,Agnetha.columns))

---
# German Cranes

Attaching the transmitters was done in a place called Grimsås, Sweden. 

Some info about the data set:
- The tracker contains a GPS sensor and a few other environmental sensors, of which the sensor data every 15 minutes (4x / hour) if an SMS message is sent via a GSM channel sent. All that sensor data can be found in the logbook.
- The color ring code of this crane is RRW-BuGBk and can be found in the column Search Studies by clicking on the small black web link Implementations, along with its tracker ID and the ring number of the metal ring with which the bird is ringed.
- Details: the most famous location is North-East France.

This datasets are obtained from https://www.movebank.org/. 

---
## Lotta
### A German Crane
---

#### Read the CSV dataset using Pandas.

In [ ]:
Lotta = pd.read_csv(
    '../../datasets/CSV/20191103_Dataset_DE_GPS_Crane_181528_iCora_Crane_15_BuBuBr-WYW_Lotta.csv')


#### Print the amount of datarows (Transmissions).

In [ ]:
len(Lotta)

#### Print the amount of columns in the dataset.

In [ ]:
len(Lotta.columns)

#### Print information related to the start position / date.

In [ ]:
print("Start date of study :" + str(Lotta[:1].timestamp)+'\n')
print("Start position (latitude) :" + str(Lotta[:1]['location-lat'])+'\n')
print("Start position (longitude) :" + str(Lotta[:1]['location-long'])+'\n')

#### Print information related to the end position / date.

In [ ]:
print("End date of study :"  + str(Lotta[-1:].timestamp)+'\n')
print("End position (latitude) :" + str(Lotta[-1:]['location-lat'])+'\n')
print("End position (longitude) :" + str(Lotta[-1:]['location-long'])+'\n')

#### Show first row of dataframe.

In [ ]:
Lotta[:1]

#### Print columnames and their datatypes.

In [ ]:
Lotta.dtypes

#### Create a table and put the values, obtained from the line above, in that table.

#### Columns with their data type (30): 

|Column|Type|Desc.|
|--|--|--|
|event-id | int| The ID of the transmission (Unique) |
|visible|bool | Is the tracker still visible or not? | 
|timestamp |datetime| The DateTime of transmission |
|location-long| float|The Longitude location |
|location-lat | float| The Latitude location |
|acceleration-raw-x|float|Speed +/- from last transmission x |
|acceleration-raw-y|float|Speed +/- from last transmission y |
|acceleration-raw-z|float|Speed +/- from last transmission z |
|bar:barometric-height |float|Pressure on the sensor. |
|battery-charge-percent|int| Battery power left in percentage.|
|battery-charging-current |float| Current running trough the battery in Ampere|
|external-temperature  |float|Temperature of the tracker measured in degrees celsius|
|gps:hdop| float| Horizontal dilution of precision|
|gps:satelitte-count |int | |
|gps-time-to-fix|float |time to acquire satellite signals in seconds.|
|ground-speed|float | Speed of the bird while on the ground. |
|heading | float| Direction the bird is faceing (In degrees) |
|height-above-msl | float| Height of the bird (above mean sea level) |
|import-marked-outlier|bool||
|gls:light-level| float|Amount of light recieved by tracker, measured in ambient light level|
|mag:magnetic-field-raw-x | float||
|mag:magnetic-field-raw-y |float||
|mag:magnetic-field-raw-z | float||
|orn:transmission-protocol | string |Type of transmission protocol |
|tag-voltage |float| Voltage level of the tracker. |
|sensor-type | string |Type of sensor used in tracker.  |
|individual-taxon-canonical-name | string | Name of the bird, in latin. |
|tag-local-identifier| int | ID of the tracker |
|individual-local-identifier | int|ID of the crane |
|study-name |string|Name of the study |

#### Sensors in sets

|X|Sensors|Desc|
|--|--|--|
|1|BAR|Baromatric sensor|
|2|GPS|Location sensor|
|3|MAG|A MEMS magnetic field sensor|
|4|ORN|A transmission protocol sensor|
|5|GLS|A light level geolocator,|


#### Info on sensors:
- Bar: Baromatric sensor used to measure the pressure on an object.
- Gps: Sensor used for data related to GPS
- Mag: A MEMS magnetic field sensor is a small-scale microelectromechanical systems (MEMS) device for detecting and measuring magnetic fields (Magnetometer). Many of these operate by detecting effects of the Lorentz force: a change in voltage or resonant frequency may be measured electronically, or a mechanical displacement may be measured optically. Compensation for temperature effects is necessary. Its use as a miniaturized compass may be one such simple example application.
- GLS: A light level geolocator, light-level logger or GLS (global location sensor) is essentially a lightweight, electronic archival tracking device, usually used in bird migration research to map migration routes, identify important staging areas, and sometimes provide additional ecological information. A geolocator periodically records ambient light level (solar irradiance) to determine location.
- Orn: Sensor used for tranmssion protocols.



#### Create a plot using MatplotLib.

In [ ]:
create_plot(Lotta,'location-long','location-lat') 

#### Create a plot MatplotLib on CartopyMap.

In [ ]:
dfs_on_cartopymap([Lotta],'location-long','location-lat','red',1)

---
## Nena
### A German Crane 

---

#### Perform basic analyses of Crane dataset and assign the returned dataframe to a variable.

In [ ]:
Nena = pd.read_csv('../../datasets/CSV/20180928_Dataset_DE_GPS_Crane_181527_iCora_Crane_13_BuBuBr-YBuBk.csv')
Nena

#### Create a plot using MatplotLib.

In [ ]:
create_plot(Nena,'location-long','location-lat') 

#### Create a plot MatplotLib on CartopyMap.

In [ ]:
dfs_on_cartopymap([Nena],'location-long','location-lat','blue',1)

### Compare the datasets to check if all the columns are the same.

In [ ]:
compare_sets(Lotta.columns,Nena.columns)


### Visualize  German Crane datasets

In [ ]:
#Create the plot and assign size of the figure
plt.figure(figsize = (20, 12))

# Create the axes of the plot and assign the projection
m = plt.axes(projection=ccrs.PlateCarree())

# Create high resolution for coastal line
m.coastlines(resolution='10m')

# Add the land with a black edge color and white fill
m.add_feature(cartopy.feature.LAND.with_scale('10m'), edgecolor='black', facecolor = "white")

# Add the sea 
m.add_feature(cfeature.OCEAN) 

# Add lakes with a black edge color
m.add_feature(cfeature.LAKES.with_scale('10m'), edgecolor = 'black')

# Add rivers
m.add_feature(cfeature.RIVERS.with_scale('10m')) 

# Add borders
m.add_feature(cfeature.BORDERS.with_scale('10m'))


# Add First Crane set (181527)
m.scatter(Nena['location-long'],Nena['location-lat'],color="red", s = 1) 

# Add Second Crane set (181527)
m.scatter(Lotta['location-long'], Lotta['location-lat'],color="blue", s = 1)


---
### Filter German crane datasets

In [ ]:
columns_to_filter = ['event-id', 'study-name',
                     'timestamp','visible',
                     'ground-speed','heading',
                     'location-long','location-lat',
                     'height-above-msl',
                     'individual-taxon-canonical-name',
                     'sensor-type','tag-voltage',                                          
                     'individual-local-identifier']

Filtered_Nena = Nena[columns_to_filter]


Filtered_Lotta = Lotta[columns_to_filter]

---
### Export German Crane datasets to JSON

Create a json file en writing the contents of the dataframe using to_json() function.

- Parameter 1: filename of file that will be created
- Parameter 2: Orientation of the data. <br>

orientation "records" makes list like [{column -> value},{column -> value},....]

In [ ]:
Filtered_Nena.to_json(
    '../../datasets/JSON/Crane_JSON/Nena-GE.json',
    orient = 'records')

Filtered_Lotta.to_json(
    '../../datasets/JSON/Crane_JSON/Lotta-GE.json',
    orient = 'records')

 ---
# Compare German and Swedish Crane datasets
---

In [ ]:
compare_sets(Lotta.columns,Agnetha.columns)

#### Swedish columns with their data type (19): 

|Column|Type|Desc.|
|--|--|--|
|event-id | int| The ID of the transmission (Unique) |
|visible|bool | Is the tracker still visible or not? | 
|timestamp |datetime| The DateTime of transmission |
|location-long| float|The Longitude location |
|location-lat | float| The Latitude location |
|argos:altitude|float | Argos type sensor for height of the transmission |
|gps:fix-type | int| Type of GPS Tracker |
|gps:hdop| float| Horizontal dilution of precision|
|gps:vdop| float| Vertical dilution of precision |
|ground-speed|float | Speed of the bird while on the ground. |
|heading | float| Direction the bird is faceing (In degrees) |
|height-above-ellipsoid | float| Height of the bird HAE -> MSL |
|tag-tech-spec |float | |
|tag-voltage|float | Voltage level of the tracker. |
|sensor-type|string | Type of sensor used in tracker. |
|individual-taxon-canonical-name | string|Name of the bird, in latin. |
|tag-local-identifier |int |ID of the tracker.  |
|individual-local-identifier |int |ID of the bird |
|study-name | string|Name of the study |



#### German columns with their data type (30): 

|Column|Type|Desc.|
|--|--|--|
|event-id | int| The ID of the transmission (Unique) |
|visible|bool | Is the tracker still visible or not? | 
|timestamp |datetime| The DateTime of transmission |
|location-long| float|The Longitude location |
|location-lat | float| The Latitude location |
|acceleration-raw-x|float|Speed +/- from last transmission x |
|acceleration-raw-y|float|Speed +/- from last transmission y |
|acceleration-raw-z|float|Speed +/- from last transmission z |
|bar:barometric-height |float|Pressure on the sensor. |
|battery-charge-percent|int| Battery power left in percentage.|
|battery-charging-current |float| Current running trough the battery in Ampere|
|external-temperature  |float|Temperature of the tracker measured in degrees celsius|
|gps:hdop| float| Horizontal dilution of precision|
|gps:satelitte-count |int | |
|gps-time-to-fix|float |time to acquire satellite signals in seconds.|
|ground-speed|float | Speed of the bird while on the ground. |
|heading | float| Direction the bird is faceing (In degrees) |
|height-above-msl | float| Height of the bird (above mean sea level) |
|import-marked-outlier|bool||
|gls:light-level| float|Amount of light recieved by tracker, measured in ambient light level|
|mag:magnetic-field-raw-x | float||
|mag:magnetic-field-raw-y |float||
|mag:magnetic-field-raw-z | float||
|orn:transmission-protocol | string |Type of transmission protocol |
|tag-voltage |float| Voltage level of the tracker. |
|sensor-type | string |Type of sensor used in tracker.  |
|individual-taxon-canonical-name | string | Name of the bird, in latin. |
|tag-local-identifier| int | ID of the tracker |
|individual-local-identifier | int|ID of the crane |
|study-name |string|Name of the study |


#### columns in both data set:

|Column|Type|Desc.|
|--|--|--|
|event-id | int| The ID of the transmission (Unique) |
|visible|bool | Is the tracker still visible or not? | 
|timestamp |datetime| The DateTime of transmission |
|location-long| float|The Longitude location |
|location-lat | float| The Latitude location |
|gps:hdop| float| Horizontal dilution of precision|
|ground-speed|float | Speed of the bird while on the ground. |
|heading | float| Direction the bird is faceing (In degrees) |
|tag-voltage |float| Voltage level of the tracker. |
|sensor-type | string |Type of sensor used in tracker.  |
|individual-taxon-canonical-name | string | Name of the bird, in latin. |
|tag-local-identifier| int | ID of the tracker |
|individual-local-identifier | int|ID of the crane |
|study-name |string|Name of the study |


#### Different columns with equalivent meaning:

|German|Swedish|
|--|--|
|height-above-msl | height-above-ellipsiod|

#### In Swedish sets but not in German:

|Column|Type|Desc.|
|--|--|--|
|acceleration-raw-x|float|Speed +/- from last transmission x |
|acceleration-raw-y|float|Speed +/- from last transmission y |
|acceleration-raw-z|float|Speed +/- from last transmission z |
|bar:barometric-height |float|Pressure on the sensor. |
|battery-charge-percent|int| Battery power left in percentage.|
|battery-charging-current |float| Current running trough the battery in Ampere|
|external-temperature  |float|Temperature of the tracker measured in degrees celsius|
|gps:satelitte-count |int | |
|gps-time-to-fix|float |time to acquire satellite signals in seconds.|
|import-marked-outlier|bool||
|gls:light-level| float|Amount of light recieved by tracker, measured in ambient light level|
|mag:magnetic-field-raw-x | float||
|mag:magnetic-field-raw-y |float||
|mag:magnetic-field-raw-z | float||
|orn:transmission-protocol | string |Type of transmission protocol |

#### In Swedish sets but not in German:

|Column|Type|Desc.|
|--|--|--|
|argos:altitude|float | Argos type sensor for height of the transmission |
|gps:fix-type | int| Type of GPS Tracker |
|gps:vdop| float| Vertical dilution of precision |
|height-above-ellipsoid | float| Height of the bird HAE -> MSL |
|tag-tech-spec |float | |


#### Sensors in Swedish

|X|Sensors|Desc|
|--|--|--|
|1|ARGOS||
|2|GPS|Location sensor|


#### Sensors in German

|X|Sensors|Desc|
|--|--|--|
|1|BAR|Baromatric sensor|
|2|GPS|Location sensor|
|3|MAG|A MEMS magnetic field sensor|
|4|ORN|A transmission protocol sensor|
|5|GLS|A light level geolocator,|



---
# Lithuanian Cranes

Attaching the transmitters was done in a place called Grimsås, Sweden. 

Some info about the data set:
- The tracker contains a GPS sensor and a few other environmental sensors, of which the sensor data every 15 minutes (4x / hour) if an SMS message is sent via a GSM channel sent. All that sensor data can be found in the logbook.
- The color ring code of this crane is RRW-BuGBk and can be found in the column Search Studies by clicking on the small black web link Implementations, along with its tracker ID and the ring number of the metal ring with which the bird is ringed.
- Details: the most famous location is North-East France.

This datasets are obtained from https://www.movebank.org/. 

---
## Lita
### A Lithuanian Crane 

---

#### Perform basic analyses of Crane dataset and assign the returned dataframe to a variable.

In [ ]:
Lita = basic_analysis('../../datasets/CSV/20200103_Dataset_LT_TrackerID_16121_Crane_Lita.csv')
Lita

#### Create a plot using MatplotLib.

In [ ]:
create_plot(Lita,'location-long','location-lat') 

#### Create a plot MatplotLib on CartopyMap.

In [ ]:
dfs_on_cartopymap([Lita],'location-long','location-lat','green',1)

In [ ]:
columns_to_filter = ['event-id', 'study-name',
                     'timestamp','visible',
                     'ground-speed','heading',
                     'location-long','location-lat',
                     'height-above-msl',
                     'individual-taxon-canonical-name',
                     'sensor-type','tag-voltage',                                          
                     'individual-local-identifier']

Filtered_Lita = Lita[columns_to_filter]

In [ ]:
Filtered_Lita[:1]

In [ ]:
Filtered_Lita.to_json(
    '../../datasets/JSON/Crane_JSON/Lita-LT.json',
    orient = 'records')

---
## TITLE
### SUBTITLE

---

#### Read the CSV dataset using Pandas.

#### Print the amount of datarows (Transmissions).

#### Print the amount of columns in the dataset.

#### Print information related to the start position / date.

#### Print information related to the end position / date.

#### Show first row of dataframe.

#### Print columnames and their datatypes.

#### Create a table and put the values, obtained from the line above, in that table.

#### Create a plot using MatplotLib.

#### Create a plot MatplotLib on CartopyMap.